In [1]:
"""
    @author: Paweł Kubiak
"""
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import time

# Load words list
words = np.load('wordsList.npy').tolist()
words = [word.decode('UTF-8') for word in words]
word_vectors = np.load('wordVectors.npy')

# Load word2vec embeding
train_x = np.load('reviews_train_x.npy')
train_y = np.load('reviews_train_y.npy')
test_x = np.load('reviews_test_x.npy')
test_y = np.load('reviews_test_y.npy')

print(train_x.shape)

# Consts
TIME_STEPS = 250
BATCH_SIZE = 256
STATE_SIZE = 200
TRAIN_STEPS = 100
LSTM_LAYERS = 3

# Data loaders
x_input = tf.placeholder(tf.int64, shape = (None, TIME_STEPS))
y_input = tf.placeholder(tf.int64, shape = (None, 2))

x_embed = tf.gather(word_vectors, x_input)
print(x_embed)
x_unpack = tf.unstack(x_embed, TIME_STEPS, 1)
print(x_unpack)

(23000, 250)
Tensor("Gather:0", shape=(?, 250, 50), dtype=float32)
[<tf.Tensor 'unstack:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'unstack:1' shape=(?, 50) dtype=float32>, <tf.Tensor 'unstack:2' shape=(?, 50) dtype=float32>, <tf.Tensor 'unstack:3' shape=(?, 50) dtype=float32>, <tf.Tensor 'unstack:4' shape=(?, 50) dtype=float32>, <tf.Tensor 'unstack:5' shape=(?, 50) dtype=float32>, <tf.Tensor 'unstack:6' shape=(?, 50) dtype=float32>, <tf.Tensor 'unstack:7' shape=(?, 50) dtype=float32>, <tf.Tensor 'unstack:8' shape=(?, 50) dtype=float32>, <tf.Tensor 'unstack:9' shape=(?, 50) dtype=float32>, <tf.Tensor 'unstack:10' shape=(?, 50) dtype=float32>, <tf.Tensor 'unstack:11' shape=(?, 50) dtype=float32>, <tf.Tensor 'unstack:12' shape=(?, 50) dtype=float32>, <tf.Tensor 'unstack:13' shape=(?, 50) dtype=float32>, <tf.Tensor 'unstack:14' shape=(?, 50) dtype=float32>, <tf.Tensor 'unstack:15' shape=(?, 50) dtype=float32>, <tf.Tensor 'unstack:16' shape=(?, 50) dtype=float32>, <tf.Tensor 'unstack:17' 

In [2]:
# print(words[2])
words[0] = ''
words[399999] = '???'
# 0 - nothing
# 399999 - unk
def vec2sent(vec):
    return ' '.join(map(lambda i: words[i], vec)).strip() 

def evaluate(pred, x, y):
    count = 0
    for i in range(len(pred)):
        if pred[i] == 0:
            count += 1
            print(vec2sent(x[i]), 'POSITIVE' if y[i][0] else 'NEGATIVE')
            if count >= 5:
                break

In [3]:
"""
  @params input
  @params learning<True/False> are we learning or testing?
  @params params<Dictionary>
"""
# def RNN(x, learning):
    
# #     lstm = tf.contrib.rnn.LSTMCell(STATE_SIZE)
    
# #     lstm = tf.contrib.rnn.MultiRNNCell(
# #         [tf.contrib.rnn.LSTMCell(STATE_SIZE) for _ in range(LSTM_LAYERS)])
#     layers = []
#     for _ in range(LSTM_LAYERS):
#         cell = tf.contrib.rnn.GRUCell(STATE_SIZE)
#         # variational_recurrent=True, input_size=STATE_SIZE, dtype=tf.float32 - pogarsza wyniki
#         wrap = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=1.0 - dropout, state_keep_prob=1.0 - 0.5*dropout)
#         layers.append(wrap)
        
#     lstm = tf.contrib.rnn.MultiRNNCell(layers)
        
#     outputs, states = tf.contrib.rnn.static_rnn(lstm, x, dtype=tf.float32)
    
#     dense = tf.layers.dense(outputs[-1], 2)
    
#     return dense



# dropout = tf.placeholder(tf.float32)

# rnn = RNN(x_unpack, dropout)
# prediction = tf.argmax(tf.nn.softmax(rnn), 1)

# loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=rnn, labels=y_input))
# #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss_op)

# global_step = tf.Variable(0, trainable=False)
# starter_learning_rate = 1e-3
# learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
#                                            90, 0.96, staircase=True)

# optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_op, global_step = global_step)

# correct_pred = tf.equal(prediction, tf.argmax(y_input, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# init = tf.global_variables_initializer()

# print(rnn)

'\n  @params input\n  @params learning<True/False> are we learning or testing?\n  @params params<Dictionary>\n'

In [4]:
# import sys

# verbose = False

# t0 = time.time()
# with tf.Session() as session:
#     session.run(init)

#     for i in range(100):
#         idx = np.arange(len(train_x))
#         np.random.shuffle(idx)
#         idx = np.array_split(idx, (len(idx)+BATCH_SIZE-1)//BATCH_SIZE)

#         accs = []
#         for step in tqdm(idx, leave = False):
# #             step = np.random.choice(len(train_x), BATCH_SIZE)
#             x_batch, y_batch = train_x[step], train_y[step]
#             _, acc = session.run([optimizer, accuracy], feed_dict = {x_input: x_batch, y_input: y_batch, dropout: 0.2})
#             accs.append(acc)
#         acc_train = 1.0*sum(accs)/len(accs)
        
#         lr, pred, acc = session.run([learning_rate, correct_pred, accuracy], feed_dict = {x_input: test_x, y_input: test_y, dropout: 0.0})
        
#         dt = time.time() - t0
#         sys.stderr.flush()
#         sys.stdout.flush()
#         l = list(map(lambda i: 0 if pred[i] else np.count_nonzero(test_x[i]), range(len(pred))))
#         l = 1.0*sum(l) / np.count_nonzero(l)
#         print("{} acc: {:.2f} acc_train: {:.2f} time: {:.2f}, lr: {}, len: {}".format(i, 100*acc, 100*acc_train, dt, lr, l))
        
# #         if i>5:
# #             evaluate(pred, test_x, test_y)
#         sys.stdout.flush()

In [ ]:
import itertools
from datetime import datetime

tf.reset_default_graph()
# Factory for different LSTM cells
def cell(ctype, state_size, dropout, state_dropout, training):
    if ctype != 'NORM':
        if ctype == 'BASIC':
            cell = tf.contrib.rnn.BasicLSTMCell(state_size)

        # Gated Recurrent Unit cell (cf. http://arxiv.org/abs/1406.1078).
        if ctype == 'GRU':
            cell = tf.contrib.rnn.GRUCell(state_size)

        # Long short-term memory unit (LSTM) recurrent network cell.
        if ctype == 'LSTM':
            cell = tf.contrib.rnn.LSTMCell(state_size)

        # Long short-term memory unit (LSTM) recurrent network cell with peephholes
        if ctype == 'LSTM-PEEP':
            cell = tf.contrib.rnn.LSTMCell(state_size, use_peepholes=True)
        
        return tf.contrib.rnn.DropoutWrapper(
            cell,
            output_keep_prob = tf.where(training, 1.0 - dropout, 1.0),
            state_keep_prob = tf.where(training, 1.0 - state_dropout, 1.0)
        )
    else:
        return tf.contrib.rnn.LayerNormBasicLSTMCell(
            state_size,
            dropout_keep_prob = tf.where(training, 1 - dropout, 1.0)
        )
    
    
def network(input, output, training, params):
    
    
    layers = []
    for _ in range(params['layer_count']):
        layers.append(cell(params['cell_type'], params['state_size'], params['dropout'], params['state_dropout'], training))
    
    lstm = tf.contrib.rnn.MultiRNNCell(layers)
    outputs, states = tf.contrib.rnn.static_rnn(lstm, input, dtype=tf.float32)
    dense = tf.layers.dense(outputs[-1], 2)
    
#     prediction = tf.argmax(tf.nn.softmax(dense), 1)
    
#     loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=dense, labels=output))
    
    return dense



def hyper_network(hparams):
    for product in itertools.product(*hparams.values()):
        tf.reset_default_graph()

        x_input = tf.placeholder(tf.int64, shape = (None, TIME_STEPS))
        y_input = tf.placeholder(tf.int64, shape = (None, 2))

        x_embed = tf.gather(word_vectors, x_input)
        x_unpack = tf.unstack(x_embed, TIME_STEPS, 1)
        
        training = tf.placeholder(tf.bool)
    
        params = {key: product[i] for i, key in enumerate(hparams.keys())}
        #print('Building: ', params)
        
        yield (product, network(x_unpack, y_input, training, params), x_input, y_input, training)

    
def hyper_evaluate(network, x_input, y_input, training, parameters):
#     tf.reset_default_graph()
    prediction = tf.argmax(tf.nn.softmax(network), 1)

    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=network, labels=y_input))

    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = 1e-3
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                               90, 0.96, staircase=True)

    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_op, global_step = global_step)

    correct_pred = tf.equal(prediction, tf.argmax(y_input, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    init = tf.global_variables_initializer()
    
    size = parameters[2]
    
    with tf.Session() as session:
        session.run(init)
#         print('Init')
        for i in range(5 + size**2):
            #print('Step: ', i)
            idx = np.arange(len(train_x))
            np.random.shuffle(idx)
            idx = np.array_split(idx, (len(idx)+BATCH_SIZE-1)//BATCH_SIZE)

            accs = []
            for step in tqdm(idx, leave=False):
    #             step = np.random.choice(len(train_x), BATCH_SIZE)
                x_batch, y_batch = train_x[step], train_y[step]
                _, acc = session.run([optimizer, accuracy], feed_dict = {x_input: x_batch, y_input: y_batch, training: True})
                accs.append(acc)
            acc_train = 1.0*sum(accs)/len(accs)
            #print(acc_train)
        
            acc = session.run([accuracy], feed_dict = {x_input: test_x, y_input: test_y, training: False})
            
            data = [datetime.now()] + list(parameters)+ [i, 100.0*acc[0], 100.0*acc_train]
            line = "\t".join(map(lambda x: str(x).strip(), data))
            print(line)
            with open('./results.tsv', 'a') as f:
                f.write(line+"\n")
        return acc


params = {
    'dropout': [0.0, 0.1, 0.15, 0.3],
    'state_dropout': [0.0, 0.1, 0.2, 0.3],
    'cell_type': ['GRU', 'LSTM', 'LSTM-PEEP', 'NORM'],
    'layer_count': [1, 2, 3],
    'state_size': [25, 50, 120, 300]
}

for x in range(5):
    print('-'*30, 'Round: ', x+1, '-'*30)
    for p, net, x, y, t in hyper_network(params):
        hyper_evaluate(net, x, y, t, p)

# print(len(list(hyper_network(params))))

# for i, net in enumerate(hyper_network(params)):
#     acc = evaluate(net)
#     print(i, acc)


------------------------------ Round:  1 ------------------------------


  0%|          | 0/90 [00:00<?, ?it/s]         

2018-02-22 21:58:12.014946	25	GRU	1	0.0	0.0	0	52.7499973774	52.1888286206


  0%|          | 0/90 [00:00<?, ?it/s]         

2018-02-22 21:58:34.530943	25	GRU	1	0.0	0.0	1	72.7999985218	60.0554202663


...

2018-02-23 08:39:34.472393	25	LSTM	3	0.0	0.0	11	52.8999984264	52.6358942853


  0%|          | 0/90 [00:00<?, ?it/s]         

2018-02-23 08:40:34.129174	25	LSTM	3	0.0	0.0	12	51.8999993801	54.3044163121


2018-02-23 08:41:33.631124	25	LSTM	3	0.0	0.0	13	54.6000003815	55.2953783009


 53%|█████▎    | 48/90 [00:49<00:43,  1.04s/it]